In [69]:
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup
import pandas as pd
import requests
from time import strftime
import datetime
import time
from pandas import ExcelWriter
import re
import pyodbc
import asyncio
from selenium.webdriver.common.action_chains import ActionChains
import numpy as np
import pdb
from ipywidgets import IntProgress
from IPython.display import display
import os
from IPython.display import clear_output
import json
from selenium.webdriver.support import expected_conditions

In [70]:
start_ = datetime.datetime.now()
fecha = datetime.date.today()
hoy = fecha.strftime('%Y/%m/%d')

In [167]:
user_agent = 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/60.0.3112.50 Safari/537.36'

chrome_options = Options()
chrome_options.add_argument("--headless")
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument(f'user-agent={user_agent}')
chrome_options.add_argument("--window-size=1825x1844")
browser = webdriver.Chrome( options = chrome_options)

In [168]:
browser.get("https://www.zaful.com/")            

In [169]:
browser.execute_script('''
$('#js-changeCountry').mouseover();
document.getElementsByClassName('header-country-wrapper')[0].getElementsByTagName('a')[0].click();
document.getElementsByClassName('header-country-wrapper')[0].click();
document.querySelector("#js-changeCountry > div > div.header-country-wrapper > div > ul.hot-country-list > li:nth-child(1) > a").click();

document.getElementsByClassName('link-update-preferences logsss_event_cl')[0].click();

''')

In [73]:
#LINKS 
links_aux = browser.execute_script('''
            var arr=[];
            var arn=[];

            var cant = document.getElementsByClassName('nav-item').length;

            for (i=0;i<cant;i++){

                arr.push(document.getElementsByClassName('nav-item')[i].getElementsByTagName('a')[0].getAttribute('href'));

                arn.push(document.getElementsByClassName('nav-item')[i].getElementsByTagName('a')[0].textContent);
            }

            function zip(a, b) {
                  var arr = [];
                  for (var key in a) arr.push([a[key], b[key]]);
                  return arr;
                }

            return zip(arr,arn);
            ''')

In [74]:
LINKS = [i for i in links_aux if i[1] in ["Women","Men"]]

In [75]:
paginacion = []
paginacion_total = []
for LINK in LINKS:
    
    browser.get(LINK[0])
    
    elementos_paginacion = browser.execute_script('''
                return document.getElementsByClassName('listspan')[0].getElementsByTagName('a');
                ''')
    paginacion_aux = []
    for page in elementos_paginacion:
        if page.text != "":
            try:
                pagina = int(page.get_attribute('data-page'))
                paginacion_aux.append(page)
            except:
                pass   
    # max page
    flag_pag = 0
    for i in paginacion_aux:
        if int(i.text) > flag_pag:
            flag_pag = int(i.text)
    
    for x in range(1,flag_pag):
        paginacion_total.append([LINK[0]+f'g_{x}.html' , LINK[1]])
            
    

In [165]:
copyL = copyL[2:].copy()

In [180]:
link

['https://www.zaful.com/clothes-e_1/g_52.html', 'Women']

In [176]:
copyL = copyL[14:].copy()

In [181]:
len(copyL)

152

In [178]:
#Declaro lista que tendrá todos los items de la web.

#items = [] 

skips=0
#loop de todos los links
for link in copyL:
    
    try:
        browser.get(link[0])
        
    except:
        pass
    
    #guardar para después hacer requests de home
    home = browser.current_url
    
    #cantidad de productos que muestra la página
    cantidad_productos = browser.execute_script('''
                            var cant = document.getElementsByClassName('js_proList_item').length;
                            return cant;
                            ''')
    style = {'description_width': 'initial'}
    test = IntProgress(style=style, min = 0, max = cantidad_productos, description = link[0].split('/g')[1]+','+link[1], bar_style='danger',)
    display(test)
            
    for index_producto in range(cantidad_productos):
        #Click en un item 
        try:
            browser.execute_script(f'''
                    document.getElementsByClassName('pic js_list_link pr logsss_event_cl')[{index_producto}].click();
                    ''')
        except:
            continue
            skips+=1
        WebDriverWait(browser, 100).until(EC.presence_of_element_located((By.CLASS_NAME,'promotion-share-layer')))

        talles = browser.find_elements_by_class_name('js-sizeItem')
        for index_talle, _ in enumerate(talles):

            #Click en el talle
            browser.execute_script(f'''document.getElementsByClassName('js-sizeItem')[{index_talle}].click();''')

            #para que cargue.
            WebDriverWait(browser, 100).until(EC.presence_of_element_located((By.CLASS_NAME,'goods-intro')))

            #Cantidad de colores
            cantidad_colores = browser.execute_script('''
                var colores =document.querySelector("#elf-mood-goods-body > section.w.clearfix.mt15.pr.elf-mood-good-wrap > div.goods-info.fr.pr > div.attr-info.mt10.clearfix.logsss_event_ps > dl.attr-item.mb10.clearfix.js-attrItem.color > dd").getElementsByTagName('p').length;
                return colores;''')

            #flag para timeout
            WebDriverWait(browser, 100).until(EC.presence_of_element_located((By.CLASS_NAME,'goods-intro')))

            #loop de colores
            for index_color in range(cantidad_colores):

                #Si el color es DISABLED, significa que no esta disponible.
                if "DISABLED" == browser.execute_script(f'''
                a = document.querySelector("#elf-mood-goods-body > section.w.clearfix.mt15.pr.elf-mood-good-wrap > div.goods-info.fr.pr > div.attr-info.mt10.clearfix.logsss_event_ps > dl.attr-item.mb10.clearfix.js-attrItem.color > dd").getElementsByTagName('p')[{index_color}].getAttribute('class').trim().toUpperCase();
                return a;'''):
                    continue
                #Click en el color
                browser.execute_script(f'''
                document.querySelector("#elf-mood-goods-body > section.w.clearfix.mt15.pr.elf-mood-good-wrap > div.goods-info.fr.pr > div.attr-info.mt10.clearfix.logsss_event_ps > dl.attr-item.mb10.clearfix.js-attrItem.color > dd").getElementsByTagName('p')[{index_color}].click();
                ''')

                #flag para timeout
                WebDriverWait(browser, 100).until(EC.presence_of_element_located((By.CLASS_NAME,'goods-intro')))

                #Sku
                sku = browser.execute_script("""var x = document.getElementsByClassName('sku')[0].getElementsByTagName('span')[0].textContent;
                                       return x;""")

                #Descripcion
                descripcion = browser.execute_script('''
                                var desc = document.getElementsByClassName('js-goods-title goods-text')[0].textContent;
                                return desc;
                                ''')

                #Precio descuento
                precio_dto = browser.execute_script('''
                            var precio_dto = document.getElementsByClassName('price-info mt10')[0].getElementsByTagName('span')[0].textContent;
                            return precio_dto;
                            ''')

                #tipologia 
                tipologia = browser.execute_script('''
                    return document.querySelector("#elf-mood-goods-body > div.path.mt20.w.logsss_event_cl > div").textContent;
                    ''')

                #Precio original
                try:
                    precio_original = browser.execute_script('''
                    var precio_ori = document.getElementsByClassName('price-info mt10')[0].getElementsByTagName('span')[1].textContent;
                    return precio_ori;''')
                except:
                    precio_original = precio_dto

                #Asignar 999 en el input y clickear en el boton del signo +.
                try:
                    time.sleep(1)
                    browser.execute_script('''
                            document.getElementsByClassName('attr-item clearfix qty')[0].getElementsByTagName('input')[0].value = 999;

                            $('#js-goodsQty > button.add.js-add.logsss_event_cl').click();
                            ''')
                except:
                    nohay = browser.execute_script('''
                        document.getElementsByClassName('process-shipping pr f14 nonsale-msg js-isnonsale-good')[0].textContent;
                        ''')

                #devuelve el stock
                try:
                    stock = browser.execute_script('''
                    function sleep(ms) {
                          return new Promise(resolve => setTimeout(resolve, ms));
                            }
                    sleep(55);
                    return $('#js-goodsQty > input[type=text]').val();
                    ''')
                except:
                    stock = nohay
                try:
                    time.sleep(1)
                    stock_aux= browser.execute_script('''return document.getElementsByClassName('js-add-notice stock-tip fb ml20')[0].textContent;''')
                except:
                    stock_aux ='posible 999'
                #talle
                talle = browser.execute_script(f'''
                        var talle = document.getElementsByClassName('js-sizeItem')[{index_talle}].textContent.trim();
                        return talle;''')

                #img producto
                img = browser.execute_script('''
                            return document.querySelector("#js-goodsGalleryShow > img").getAttribute('src');
                            ''')
                url_producto = browser.current_url
                items.append([sku,descripcion,link[1],talle,precio_dto,precio_original,tipologia,stock,stock_aux,img,url_producto])
                
                
        test.value +=1
        test.description = f"± {int(cantidad_productos)*15}: {test.value} / {cantidad_productos}"
        
        try:
            #jquery para clickear en el "home" de Women
            browser.execute_script(f'''
            $(".path.mt20.w.logsss_event_cl a span:contains('{link[1]}')").click();
            ''')
        except:
            print(f"No se pudo hacer click en {link[1]}")
            pass

    test.bar_style='success'
    clear_output() 

    

IntProgress(value=0, bar_style='danger', description='_52.html,Women', max=60, style=ProgressStyle(description…

WebDriverException: Message: no such session
  (Driver info: chromedriver=2.41.578700 (2f1ed5f9343c13f73144538f15c00b370eda6706),platform=Linux 4.15.0-66-generic x86_64)


In [118]:
len(items)

24933

In [86]:
print(browser.current_url)

https://latam.zaful.com/pantalones-cortos-con-cinturon-a-rayas-de-cintura-alta-p_709076.html


In [85]:
browser.save_screenshot('asd.png')

True

In [303]:
browser.quit()

In [269]:
index_producto

41

In [256]:
hoja

2

In [248]:
Message: unknown error: session deleted because of page crash
from unknown error: cannot determine loading status
from tab crashed
  (Session info: headless chrome=74.0.3729.131)
  (Driver info: chromedriver=2.41.578700 (2f1ed5f9343c13f73144538f15c00b370eda6706),platform=Linux 4.15.0-66-generic x86_64)


3

In [239]:
cantidad_productos

60

In [251]:
asd = items

In [24]:
browser.save_screenshot('asd.png')

True

In [187]:
browser.save_screenshot('asd.png')

True

In [10]:
df = pd.DataFrame(items)

In [11]:
df.rename(columns={0:'id_producto',
                  1:'descripcion',
                  2:'talle',
                  3:'precio_dto',
                  4:'precio_original',
                  5:'breadcrumb',
                  6:'stock',
                  7:'posible_stock',
                  8:'img',
                  9:'url'
                  },inplace=True)

In [12]:
df

,id_producto,descripcion,talle,precio_dto,precio_original,breadcrumb,stock,posible_stock,img,url,10
0,459492601,Twist Asymmetric Crop Sweater - Red Wine S,Women,S,$15.99,RRP $27.29,Sweaters / Twist Asymmetric Crop Sweater,640,Stock less than 640,https://gloimg.zafcdn.com/zaful/pdm-product-pi...,https://www.zaful.com/twist-asymmetric-crop-sw...
1,459492616,Twist Asymmetric Crop Sweater - Dark Forest Gr...,Women,S,$15.99,RRP $27.29,Sweaters / Twist Asymmetric Crop Sweater,655,Stock less than 655,https://gloimg.zafcdn.com/zaful/pdm-product-pi...,https://www.zaful.com/twist-asymmetric-crop-sw...
2,459492607,Twist Asymmetric Crop Sweater - Gray Cloud S,Women,S,$15.99,RRP $27.29,Sweaters / Twist Asymmetric Crop Sweater,843,Stock less than 843,https://gloimg.zafcdn.com/zaful/pdm-product-pi...,https://www.zaful.com/twist-asymmetric-crop-sw...
3,459492604,Twist Asymmetric Crop Sweater - Light Khaki S,Women,S,$15.99,RRP $27.29,Sweaters / Twist Asymmetric Crop Sweater,765,Stock less than 765,https://gloimg.zafcdn.com/zaful/pdm-product-pi...,https://www.zaful.com/twist-asymmetric-crop-sw...
4,459492619,Twist Asymmetric Crop Sweater - Light Sky Blue S,Women,S,$15.99,RRP $27.29,Sweaters / Twist Asymmetric Crop Sweater,999,Stock less than 1060,https://gloimg.zafcdn.com/zaful/pdm-product-pi...,https://www.zaful.com/twist-asymmetric-crop-sw...
5,459492601,Twist Asymmetric Crop Sweater - Red Wine S,Women,S,$15.99,RRP $27.29,Sweaters / Twist Asymmetric Crop Sweater,724,Stock less than 724,https://gloimg.zafcdn.com/zaful/pdm-product-pi...,https://www.zaful.com/twist-asymmetric-crop-sw...
6,459492613,Twist Asymmetric Crop Sweater - Rose S,Women,S,$15.99,RRP $27.29,Sweaters / Twist Asymmetric Crop Sweater,764,Stock less than 764,https://gloimg.zafcdn.com/zaful/pdm-product-pi...,https://www.zaful.com/twist-asymmetric-crop-sw...
7,459492610,Twist Asymmetric Crop Sweater - White S,Women,M,$15.99,RRP $27.29,Sweaters / WHITE SweaterWHITE Sweater/Twist A...,640,Stock less than 640,https://gloimg.zafcdn.com/zaful/pdm-product-pi...,https://www.zaful.com/twist-asymmetric-crop-sw...
8,459492616,Twist Asymmetric Crop Sweater - Dark Forest Gr...,Women,M,$15.99,RRP $27.29,Sweaters / Twist Asymmetric Crop Sweater,655,Stock less than 655,https://gloimg.zafcdn.com/zaful/pdm-product-pi...,https://www.zaful.com/twist-asymmetric-crop-sw...
9,459492607,Twist Asymmetric Crop Sweater - Gray Cloud S,Women,M,$15.99,RRP $27.29,Sweaters / Twist Asymmetric Crop Sweater,843,Stock less than 843,https://gloimg.zafcdn.com/zaful/pdm-product-pi...,https://www.zaful.com/twist-asymmetric-crop-sw...


In [118]:
df[1].apply(lambda x : x.split('-')[0])

0      Plunge Colorblock Twist Sweater 
1      Plunge Colorblock Twist Sweater 
2      Plunge Colorblock Twist Sweater 
3      Plunge Colorblock Twist Sweater 
4      Plunge Colorblock Twist Sweater 
5      Plunge Colorblock Twist Sweater 
6      Plunge Colorblock Twist Sweater 
7      Plunge Colorblock Twist Sweater 
8      Plunge Colorblock Twist Sweater 
9      Plunge Colorblock Twist Sweater 
10     Plunge Colorblock Twist Sweater 
11     Plunge Colorblock Twist Sweater 
12     Plunge Colorblock Twist Sweater 
13     Plunge Colorblock Twist Sweater 
14     Plunge Colorblock Twist Sweater 
15     Plunge Colorblock Twist Sweater 
16     Plunge Colorblock Twist Sweater 
17     Plunge Colorblock Twist Sweater 
18     Plunge Colorblock Twist Sweater 
19     Plunge Colorblock Twist Sweater 
20     Plunge Colorblock Twist Sweater 
21       Twist Asymmetric Crop Sweater 
22       Twist Asymmetric Crop Sweater 
23       Twist Asymmetric Crop Sweater 
24       Twist Asymmetric Crop Sweater 


In [178]:
df

,0,1,2,3,4,5,6,7,8,9
0,459492001,Plunge Colorblock Twist Sweater - Multi-a S,S,$18.99,RRP $31.59,Sweaters / Plunge Colorblock Twist Sweater,965,Stock less than 965,https://gloimg.zafcdn.com/zaful/pdm-product-pi...,https://www.zaful.com/colorblock-plunging-neck...
1,459492001,Plunge Colorblock Twist Sweater - Multi-a S,S,$18.99,RRP $31.59,Sweaters / Plunge Colorblock Twist Sweater,616,Stock less than 616,https://gloimg.zafcdn.com/zaful/pdm-product-pi...,https://www.zaful.com/plunge-colorblock-twist-...
2,459492004,Plunge Colorblock Twist Sweater - Multi-b S,S,$18.99,RRP $31.59,Sweaters / Plunge Colorblock Twist Sweater,878,Stock less than 878,https://gloimg.zafcdn.com/zaful/pdm-product-pi...,https://www.zaful.com/plunge-colorblock-twist-...
3,459492007,Plunge Colorblock Twist Sweater - Multi-c S,S,$18.99,RRP $31.59,Sweaters / Plunge Colorblock Twist Sweater,959,Stock less than 959,https://gloimg.zafcdn.com/zaful/pdm-product-pi...,https://www.zaful.com/plunge-colorblock-twist-...
4,459492010,Plunge Colorblock Twist Sweater - Multi-d S,S,$18.99,RRP $31.59,Sweaters / Plunge Colorblock Twist Sweater,857,Stock less than 857,https://gloimg.zafcdn.com/zaful/pdm-product-pi...,https://www.zaful.com/plunge-colorblock-twist-...
5,459492013,Plunge Colorblock Twist Sweater - Multi-e S,S,$18.99,RRP $31.59,Sweaters / Plunge Colorblock Twist Sweater,898,Stock less than 898,https://gloimg.zafcdn.com/zaful/pdm-product-pi...,https://www.zaful.com/plunge-colorblock-twist-...
6,459492016,Plunge Colorblock Twist Sweater - Multi-f S,S,$18.99,RRP $31.59,Sweaters / Plunge Colorblock Twist Sweater,773,Stock less than 773,https://gloimg.zafcdn.com/zaful/pdm-product-pi...,https://www.zaful.com/plunge-colorblock-twist-...
7,459492019,Plunge Colorblock Twist Sweater - Multi-g S,M,$18.99,RRP $31.59,Sweaters / Plunge Colorblock Twist Sweater,965,Stock less than 965,https://gloimg.zafcdn.com/zaful/pdm-product-pi...,https://www.zaful.com/colorblock-plunging-neck...
8,459492001,Plunge Colorblock Twist Sweater - Multi-a S,M,$18.99,RRP $31.59,Sweaters / Plunge Colorblock Twist Sweater,616,Stock less than 616,https://gloimg.zafcdn.com/zaful/pdm-product-pi...,https://www.zaful.com/plunge-colorblock-twist-...
9,459492004,Plunge Colorblock Twist Sweater - Multi-b S,M,$18.99,RRP $31.59,Sweaters / Plunge Colorblock Twist Sweater,878,Stock less than 878,https://gloimg.zafcdn.com/zaful/pdm-product-pi...,https://www.zaful.com/plunge-colorblock-twist-...


In [159]:
print(df[9][5])

https://www.zaful.com/plunge-colorblock-twist-sweater-p_818115.html
